In [17]:
from ct_support_code import *
import numpy as np
data = np.load('ct_data.npz')
X_train = data['X_train']
X_val = data['X_val']
X_test = data['X_test']
y_train = data['y_train']
y_val = data['y_val']
y_test = data['y_test']


In [18]:
#Q1.a NEWEST VERSION
# print("mean of y_train: ",np.mean(y_train))
#1.a
# standard error = sigma/sqrt(N) = sqrt(variance/N) 
# report mean with a standard error assuming that each entry is independent
N_yvar = y_val.shape[0]
mean_yval = np.mean(y_val)
mean_ytrn = np.mean(y_train)
std_err_yval = np.sqrt(np.var(y_val)/N_yvar)
std_err_ytrn = np.sqrt(np.var(y_train[:N_yvar])/N_yvar)
print("mean of y_train: ",mean_ytrn,", with standard error:",std_err_ytrn)
print("mean of y_validate: ",mean_yval,", with standard error:",std_err_yval)


# Standard Error bar only give an indication of 
# how precisely we think we have measured the mean of the distribution with our  samples.

mean of y_train:  -9.13868774539957e-15 , with standard error: 0.011926272462733948
mean of y_validate:  -0.2160085093241599 , with standard error: 0.012903383410668332


In [ ]:
# Q1.b NEWEST VERSION
# X_train = np.array([[1,2,3,3,3,5,6,7,8,9,3,5],
#               [2,2,4,4,6,7,8,9,0,1,3,6],
#               [3,2,5,5,7,8,9,0,8,2,3,7],
#               [2,2,4,4,6,7,8,9,0,1,3,6]])
# X_test = np.array([[1,2,3,3,3,5,6,7,8,9,3,5],
#               [2,2,4,4,6,7,8,9,0,1,3,6],
#               [3,2,5,5,7,8,9,0,8,2,3,7],
#               [2,2,4,4,6,7,8,9,0,1,3,6]])
# X_val = np.array([[1,2,3,3,3,5,6,7,8,9,3,5],
#               [2,2,4,4,6,7,8,9,0,1,3,6],
#               [3,2,5,5,7,8,9,0,8,2,3,7],
#               [2,2,4,4,6,7,8,9,0,1,3,6]])
print("Size of X_train Before:",X_train.shape[0],X_train.shape[1])
print("Size of X_val Before:",X_val.shape[0],X_val.shape[1])
print("Size of X_test Before:",X_test.shape[0],X_test.shape[1])

#1 Constant value
Xtrain_f_var = np.array(np.var(X_train,axis=0), dtype=bool)
Xtrain_feature_removed = np.delete([i for i in range(X_train.shape[1])],Xtrain_f_var)
print("Index of Xtrain's constant feature discarded = ",Xtrain_feature_removed)
X_train = X_train[:,Xtrain_f_var]
#2 Duplicate
_,index_xtrn = np.unique(X_train, return_index=True, axis=1)
Xtrain_sample_removed = np.delete([i for i in range(X_train.shape[1])],index_xtrn)
X_train=X_train[:,np.sort(index_xtrn)]
print("Index of Xtrain's duplicate feature discarded = ",Xtrain_sample_removed)

X_val = (X_val[:,Xtrain_f_var])[:,np.sort(index_xtrn)]
X_test = (X_test[:,Xtrain_f_var])[:,np.sort(index_xtrn)]

print("Size of X_train After:",X_train.shape[0],X_train.shape[1])
print("Size of X_val After:",X_val.shape[0],X_val.shape[1])
print("Size of X_test After:",X_test.shape[0],X_test.shape[1])

print(X_train)

In [ ]:
def RMSE(w,b,X,y_predicted):
    y_actual = np.sum(w*X,axis=1)-b
    MSE = np.square(np.subtract(y_actual,y_predicted)).mean() 
    RMSE = np.sqrt(MSE)
    return RMSE


#Q2 NEWEST VERSION
def fit_linreg(X, yy, alpha):
    # Rank+1: for every sample [x1,x2,...,xn]*[w1,w2,...,wn] + b = [x1,x2,...,xn,1]*[w1,w2,...,wn,b]
    X_bar = np.vstack([X.T,np.ones(X.shape[0])]).T
    # Add K rows to matrix of input features
    # Last element in the Kth row should be 0: leave bias b not regularized
    temp2 = np.concatenate([np.sqrt(alpha)*np.ones(X_bar.shape[1]-1),[0]])
    X_bar2 = np.vstack([X_bar, temp2])
    # Add K rows to vector of labels
    yy_bar = np.concatenate([yy,[0]])
    output= np.linalg.lstsq(X_bar2,yy_bar,rcond=None)[0]
    return output


output1 = fit_linreg(X_train,y_train,30)
w_regularized = output1[0:-1]
b_regularized = output1[-1]
output2 = fit_linreg_gradopt(X_train,y_train,30)
w_gradbased = output2[0]
b_gradbased = output2[1]

def q2a(setname,X,Y):  
    print("RMSE on",setname,"for regularization = ",RMSE(w_regularized,b_regularized,X,Y))
    print("RMSE on",setname,"for gradient based = ",RMSE(w_gradbased,b_gradbased,X,Y))
    
q2a("training set",X_train,y_train)
q2a("validation set",X_val,y_val)

In [ ]:
#Q3 HELPER FUNCTION
def fit_logreg_gradopt(X, yy, alpha):
    D = X.shape[1]
    args = (X, yy, alpha)
    init = (np.zeros(D), np.array(0))
    ww, bb = minimize_list(logreg_cost, init, args)
    return ww, bb

In [ ]:
# Q3 Newest version

K = 20 # number of thresholded classification problems to fit
mx = np.max(y_train)
mn = np.min(y_train)
hh = (mx-mn)/(K+1)
probs = np.empty((0,X_train.shape[1]+1))
thresholds = np.linspace(mn+hh, mx-hh, num=K, endpoint=True)
y_train_binary = np.empty((0,len(y_train)))
for kk in range(K):
    labels = y_train > thresholds[kk]
    y_train_binary = np.row_stack((y_train_binary,labels))
    # ... fit logistic regression to these labels
    weight,bias = fit_logreg_gradopt(X_train,labels,30)
    weight = np.append(weight,bias)
    probs = np.row_stack((probs,weight))
# shape of probs = (20,374)
# column_stack a column of X_train with ones to time with the bias
X_trn_temp = np.column_stack((X_train,np.ones(X_train.shape[0])))
X_trn_T = np.matmul(X_trn_temp,np.array(probs).T)
# X_trn_t.shape = (40754, 20)
X_val_temp = np.column_stack((X_val,np.ones(X_val.shape[0]).T))
X_val_T = np.matmul(X_val_temp,np.array(probs).T)
output3 = fit_linreg(X_trn_T,y_train,30)
ww = output3[0:20]
bb = output3[20]

RMSE_trn = RMSE(ww,bb,X_trn_T,y_train)
RMSE_val = RMSE(ww,bb,X_val_T,y_val)
print("RMSE on Training_Set = ",RMSE_trn)
print("RMSE on Validation_Set = ",RMSE_val)

#RMSE on Training_Set =  2.769307983017384
#RMSE on Validation_Set =  2.608896156668959

In [ ]:
# Q4 Newest
def initialize_parameters_random(layers_dims):
    ww = 0.1* np.random.randn(layers_dims[1]) / np.sqrt(layers_dims[1])
    bb = np.zeros(layers_dims[2])
    V = 0.1 * (np.random.randn(layers_dims[0], layers_dims[1]) / np.sqrt(layers_dims[0])).T
    bk = np.zeros(layers_dims[1])
    # print("ww.shape=",ww.shape,"bb.shape=",bb.shape,"V.shape=",V.shape,"bk,shape=",bk.shape)
    return [ww, bb, V, bk]

def fit_nn_gradopt(X, yy, alpha,layers_dims):
    D = X.shape[1]
    args = (X, yy, alpha)
    init = initialize_parameters_random(layers_dims)
    # init = (np.zeros(D), np.array(0))
    ww, bb, V, bk = minimize_list(nn_cost, init, args)
    return ww, bb, V, bk

def question4(X_train,y_train,X_val,y_val,X_test,y_test,alpha):
    K = 20
    layers_dims = [X_train.shape[1], K, 1]
    parameters_a = fit_nn_gradopt(X_train, y_train, alpha, layers_dims)
    print("*********************finished backward parameter generated**********************")

    RMSE_train_a = np.sqrt(nn_cost(parameters_a, X_train,y_train,30)[0])
    RMSE_val_a = np.sqrt(nn_cost(parameters_a, X_val,y_val,30)[0])
    RMSE_test_a = np.sqrt(nn_cost(parameters_a, X_test,y_test,30)[0])

    parameters_b = question3(X_train,y_train,X_val,y_val,X_test,y_test)
    RMSE_train = np.sqrt(nn_cost(parameters_b, X_train, y_train, 30)[0])
    RMSE_val_b = np.sqrt(nn_cost(parameters_b, X_val, y_val, 30)[0])
    RMSE_test_b = np.sqrt(nn_cost(parameters_b, X_test, y_test, 30)[0])
    
    print("For a) On the train set:", RMSE_train_a)
    print("For a) On the validation set:", RMSE_val_a)
    print("For a) On the test set:", RMSE_test_a)
    print("For b) On the train set:", RMSE_train_b)
    print("For b) On the validation set:", RMSE_val_b)
    print("For b) On the test set:", RMSE_test_b)

    return parameters_a

question4(X_train,y_train,X_val,y_val,X_test,y_test,30)

In [ ]:
#Q5 Newest
def train_nn_reg(X_train,y_train,X_val,y_val,alpha):
    K = 20
    layers_dims = [X_train.shape[1], K, 1]
    parameters = fit_nn_gradopt(X_train, y_train, alpha, layers_dims)
    RMSE_val = np.sqrt(nn_cost(parameters, X_val,y_val,30)[0])
    return RMSE_val

def maxPI(test_loc,ob_loc,bl_alpha,X_train,y_train,X_val,y_val):
    ob_val = []
    blRMSE = np.log(train_nn_reg(X_train,y_train,X_val,y_val,30))
    for loc in ob_loc：
        n1RMSE = b1RMSE - np.log(train_nn_reg(X_train,y_train,X_val,y_val,bl_alpha))
        ob_val.append(n1RMSE)
    rest_cond_mu, rest_cond_cov = gp_post_var(test_loc,ob_loc,ob_val)
    bla_mu,_ = gp_post_var(test_loc,bl_alphal,b1RMSEB V)
    max_obs = max(ob_val)
    outputs = (rest_cond_mu - max_obs)/np.diagonal(rest_cond_cov)
    next_alpha = max(outputs)
    return alpha

alphas = np.arange(0, 50, 0.02)
alpha_chosen = np.random.choice(alphas,3)
alpha_remain = np.remove(alphas,alpha_remain)
result_y = []
blRMSE = np.log(train_nn_reg(X_train,y_train,X_val,y_val,30))
